In [1]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

In [3]:
DIR="..\\..\\chunk\\18\\"
checks = pd.read_csv(f"{DIR}data.csv")
checks.drop(['name'], axis=1, inplace=True)

with open(f"{DIR}Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

checks.head()

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\191298464.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  checks = pd.read_csv(f"{DIR}data.csv")


,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00"


In [4]:
# Создадим ключевую колонку: время + касса + магазин
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


In [5]:
checks.dropna(inplace=True)
checks = checks[checks['sale_date_date'].str.startswith("20")]
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

# Достаточно отсортировать и сделать трейн-тест, чтобы построить валидацию. Забьем на разорванные сессии
train, test = train_test_split(checks, test_size=0.3)

In [7]:
# Обучим простые контетные рекомендации

# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

sentences = [preprocess_txt(k) for k in product_dict.keys()]

# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, которые оенил пользователь
# Обучим Fasttext и заэмбедим фильмы
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> фильм, но и фильм-> id, чтобы быстрее находить эмбеддинги айтемов
reverse_index_map = {}
counter = 0

for name in tqdm_notebook(product_dict.keys()):
    n_ft = 0
    reverse_index_map[counter] = name
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt(name):
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_names.build(10)

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1661400268.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(product_dict.keys()):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [8]:
reverse_item_mapping = {v: k for k, v in product_dict.items()}

def recommend(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector[reverse_index_map[iname]])
    return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [9]:
recommend(['52539', '110342'])

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\3531675664.py:12: RuntimeWarning: invalid value encountered in true_divide
  return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


['(35743) Шприцы одноразовые с иглой 5мл N1 762',
 '(56964) Линекс пор. лиоф. капс. №32 600',
 '(63827) Аркоксиа табл. п.п.о.90мг №7 647',
 '(87309) Минеральная вода Винцентка 0,7л 569',
 '(104883) Бальзам "Дикуля" Форте при заболеваниях позвоночника и суставов 75мл 718',
 '(67898) Фламакс форте табл. п.п.о 100мг N20 397',
 '(49051) Лавомакс таб.п.о.125мг №6 705',
 '(122344) Зизу Салфетки влажные для снятия макияжа с мицеллярной водой №10 496',
 '(180072) Норева Ксеродиан Плюс Крем для лица и тела пенящийся очищающий 500мл 496',
 '(56035) Фиточай Стевия 1г №20 611']

In [10]:
# Обучение эмбеддингов по последовательности
from gensim.models import Word2Vec

train['product_id'] = train['product_id'].apply(int).apply(str)

grouped = train.groupby('key_col')
sentences = []

In [11]:
# Сформируем последовательности

from tqdm import tqdm 
sentences = []
for group in tqdm_notebook(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentences.append(list(products))

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1066363157.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(grouped.groups):


  0%|          | 0/5457574 [00:00<?, ?it/s]

In [13]:
modelW2V = Word2Vec(sentences, vector_size=10)

In [14]:
sentences[0]

['115617', '142296', '37070']

In [16]:
modelW2V.wv.similar_by_vector

<bound method KeyedVectors.similar_by_vector of <gensim.models.keyedvectors.KeyedVectors object at 0x000001E477776160>>

In [19]:
def recommend_w2v(items_list):
    current_vector = np.zeros(10)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in modelW2V.wv:
            c -= 1
            continue
        current_vector += np.array(modelW2V.wv[reverse_index_map[iname]])
    return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]

In [20]:
recommend_w2v(['99821', '138583', '45321', '134475'])

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1051936176.py:10: RuntimeWarning: invalid value encountered in true_divide
  return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]


['(181271) Зубная паста Колгейт Древние секреты Безупречная Свежесть 75мл 529',
 '(67162) Габагамма капс. 300мг №50 641',
 '(187610) Гиалрипайер-10 Хондрорепарант гель гиалуроновый д/инъек.фл.0,8% 5мл №1 646',
 '(106823) ЛОкситан Шампунь Сила и Густота 300мл 476',
 '(192300) PL Перчатки латексные нестерильные р.M №10 675',
 '(67740) Аторвастатин таб.п.п.о.20мг №30 685',
 '(66406) Ополаскиватель д/полости рта Лакалют Уайт 300мл -1',
 '(35332) СТИКС Крем "Авокадо" 50 мл, уп №1 (150) 498',
 '(3434) Чай "Волшебная сила" [фильтр-пакеты 2г] N20 -1',
 '(183248) Катетер-баллон "Фолея" 2-х ходовой (№18) №1 761']